In [5]:
# =========================
# User Config (edit here)
# =========================

# 1) OAuth 凭据文件（从 Google Cloud Console 下载的 JSON）
CREDENTIALS_FILE = "client_secret_494548015190-rgm4bn6qnv12sugdhenlcpvs6vghur99.apps.googleusercontent.com.json"

# 2) token 缓存（首次授权后会自动生成）
TOKEN_FILE = "token.json"

# 3) Gmail 用户（读取自己的邮箱通常用 "me"）
USER_ID = "me"

# 4) 拉取最新多少封
MAX_RESULTS = 5  # N

# 5) Gmail 搜索语法过滤（None 表示不过滤）
# 示例：
#   QUERY = 'newer_than:7d'
#   QUERY = 'from:amazon.co.jp subject:"領収書"'
#   QUERY = 'has:attachment filename:pdf'
QUERY = None

# 6) 正文显示截断长度
BODY_PREVIEW_CHARS = 800

# 7) OAuth Scopes（只读）
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


In [2]:
!pip -q install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [6]:
import os
import base64
from typing import Optional, Tuple, List, Dict

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build


def get_gmail_service(
    credentials_file: str = CREDENTIALS_FILE,
    token_file: str = TOKEN_FILE,
    scopes: List[str] = SCOPES,
):
    """
    Jupyter-friendly Gmail API service builder.
    First run: opens browser for OAuth, saves token.json.
    """
    creds = None
    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, scopes)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists(credentials_file):
                raise FileNotFoundError(
                    f"Missing {credentials_file}. Put OAuth Desktop credentials JSON in the notebook directory."
                )
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, scopes)
            # For local notebook: will open browser and use a local redirect server
            creds = flow.run_local_server(port=0)
        with open(token_file, "w", encoding="utf-8") as f:
            f.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)


def _b64url_decode(data: str) -> bytes:
    # Gmail uses base64url, often missing padding
    data += "=" * (-len(data) % 4)
    return base64.urlsafe_b64decode(data.encode("utf-8"))


def _get_header(headers: List[Dict[str, str]], name: str) -> str:
    n = name.lower()
    for h in headers:
        if h.get("name", "").lower() == n:
            return h.get("value", "")
    return ""


def _extract_parts(payload: Dict) -> Tuple[Optional[str], Optional[str], List[Dict]]:
    """
    Return (text/plain, text/html, attachments_meta)
    """
    text_body = None
    html_body = None
    attachments = []

    def walk(part: Dict):
        nonlocal text_body, html_body, attachments

        mime_type = part.get("mimeType", "")
        filename = part.get("filename", "") or ""
        body = part.get("body", {}) or {}

        attachment_id = body.get("attachmentId")
        size = body.get("size", 0)

        # Attachments (including inline images) usually have filename or attachmentId
        if filename or attachment_id:
            if attachment_id:
                attachments.append(
                    {
                        "filename": filename or "(no-filename)",
                        "mimeType": mime_type,
                        "size": size,
                        "attachmentId": attachment_id,
                    }
                )
        else:
            data = body.get("data")
            if data:
                raw = _b64url_decode(data)
                try:
                    decoded = raw.decode("utf-8")
                except UnicodeDecodeError:
                    decoded = raw.decode("utf-8", errors="replace")

                if mime_type == "text/plain" and text_body is None:
                    text_body = decoded
                elif mime_type == "text/html" and html_body is None:
                    html_body = decoded

        for p in part.get("parts", []) or []:
            walk(p)

    walk(payload)
    return text_body, html_body, attachments


def list_latest_messages(service, user_id: str = USER_ID, max_results: int = MAX_RESULTS, q: Optional[str] = QUERY):
    resp = (
        service.users()
        .messages()
        .list(userId=user_id, maxResults=max_results, q=q)
        .execute()
    )
    return resp.get("messages", [])


def read_message_full(service, msg_id: str, user_id: str = USER_ID):
    msg = (
        service.users()
        .messages()
        .get(userId=user_id, id=msg_id, format="full")
        .execute()
    )

    payload = msg.get("payload", {}) or {}
    headers = payload.get("headers", []) or []

    subject = _get_header(headers, "Subject")
    from_ = _get_header(headers, "From")
    date_ = _get_header(headers, "Date")

    text_body, html_body, attachments = _extract_parts(payload)

    return {
        "id": msg_id,
        "threadId": msg.get("threadId"),
        "internalDate": msg.get("internalDate"),  # ms timestamp as string
        "subject": subject,
        "from": from_,
        "date": date_,
        "snippet": msg.get("snippet", ""),
        "text": text_body,
        "html": html_body,
        "attachments": attachments,
    }


In [7]:
service = get_gmail_service()

msgs = list_latest_messages(service, max_results=MAX_RESULTS, q=QUERY)

print(f"Found {len(msgs)} messages (max_results={MAX_RESULTS}, q={QUERY!r})")

for i, m in enumerate(msgs, 1):
    data = read_message_full(service, m["id"])
    print("=" * 100)
    print(f"[{i}] Subject: {data['subject']}")
    print(f"    From:    {data['from']}")
    print(f"    Date:    {data['date']}")
    print(f"    Snippet: {data['snippet']}")
    print(f"    Attachments: {len(data['attachments'])}")
    for a in data["attachments"]:
        print(f"      - {a['filename']} ({a['mimeType']}, {a['size']} bytes, attachmentId={a['attachmentId']})")

    body = data["text"] or data["html"] or ""
    if body:
        print("\n--- BODY (preview) ---")
        print(body[:BODY_PREVIEW_CHARS])
    else:
        print("\n--- BODY --- (empty)")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=494548015190-rgm4bn6qnv12sugdhenlcpvs6vghur99.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63674%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=cYzqRsc2iskdah3XEHtPeoW9LwDsLa&access_type=offline


KeyboardInterrupt: 

In [2]:
import requests
import time

# ========= 基础配置 =========
API_TOKEN = "YNndD5WeM3UFO32RKdrogf2p4hNVPlZE3r3rLCoHD3B4idpJcjyqRJbNndXM"
API_URL = "https://api.webscraper.io/api/v1/scraping-job"

SITEMAP_ID = 1423655

HEADERS = {
    "Content-Type": "application/json"
}

# ========= 硬编码的 start_urls =========
START_URLS = [
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792919170=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926365=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926450=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926715=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926800=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926951=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792927054=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792927290=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792932470=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792938055=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792939396=&search.x=62&search.y=13&startingUrlPatten=&locale=ja"
]

# ========= 循环发送请求 =========
responses = []

for idx, url in enumerate(START_URLS, start=1):
    payload = {
        "sitemap_id": SITEMAP_ID,
        "driver": "fulljs",               # 或 "fulljs"
        "page_load_delay": 2000,
        "request_interval": 2000,
        "proxy": "datacenter-us",
        "start_urls": [url],            # 每次只发一个
        "custom_id": f"custom-job-{idx}"
    }

    print(f"[{idx}] Sending scraping job for:", url)

    resp = requests.post(
        API_URL,
        params={"api_token": API_TOKEN},
        json=payload,
        headers=HEADERS,
        timeout=30
    )

    responses.append(resp)

    print("Status:", resp.status_code)
    print("Response:", resp.text)
    print("-" * 60)

    # 防止触发 API 限流（可按需要调整）
    time.sleep(6)

[1] Sending scraping job for: https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792919170=&search.x=62&search.y=13&startingUrlPatten=&locale=ja
Status: 200
Response: {"success":true,"data":{"id":38001597,"custom_id":"custom-job-1"}}
------------------------------------------------------------
[2] Sending scraping job for: https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926365=&search.x=62&search.y=13&startingUrlPatten=&locale=ja
Status: 200
Response: {"success":true,"data":{"id":38001602,"custom_id":"custom-job-2"}}
------------------------------------------------------------
[3] Sending scraping job for: https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926450=&search.x=62&search.y=13&startingUrlPatten=&locale=ja
Status: 200
Response: {"success":true,"data":{"id":38001609,"custom_id":"custom-job-3"}}
------------------------------------------------------------
[4] Sending scraping job for: https://trackings.post.japan

In [ ]:
import requests
import time

# ========= 基础配置 =========
API_TOKEN = "YNndD5WeM3UFO32RKdrogf2p4hNVPlZE3r3rLCoHD3B4idpJcjyqRJbNndXM"
API_URL = "https://api.webscraper.io/api/v1/scraping-job"

SITEMAP_ID = 1423655

HEADERS = {
    "Content-Type": "application/json"
}

# ========= 硬编码的 start_urls =========
START_URLS = [
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655385054",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655394596",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655385032",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655394585",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655384995",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655394703",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655375265",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655375302",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655394736",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090673",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654079392",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043843",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654079230",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654079355",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042535",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654083183",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090286",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654096855",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042885",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094184",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089380",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654098126",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089520",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042734",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089542",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042745",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045851",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089376",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043751",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043736",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090625",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045906",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468685",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468803",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468840",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045173",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654093134",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654079215",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041640",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090835",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654092530",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090894",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043946",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045033",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044985",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044974",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094151",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654088890",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041975",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097905",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042233",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044532",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654057062",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654093764",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654084524",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097371",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654083942",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654082726",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654076500",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654076371",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483653946565",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654087630",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654077981",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654084491",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654091642",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045711",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042701",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654098244",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089715",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045744",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654098266",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094604",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094674",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094486",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090695",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654091922",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090603",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654091443",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097496",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097220",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654091255",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094932",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468851",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043740",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097570",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043972",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041651",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044742",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654092585",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926450=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090850",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043950",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090861",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513935",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926951=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044075",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042642",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042384",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654096870",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045766",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089273",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042830",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468965",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792938055=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468663",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654095046",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097113",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654092876",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090791",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044635",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654092084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468954",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468943",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094906",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043213",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043202",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654080044",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090721",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044602",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042955",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089704",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792927290=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042970",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094560",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045840",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094571",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043025",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094615",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043036",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043014",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654093160",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044823",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097054",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090205",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043191",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094744",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468862",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791469691",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791469676",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654080884",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654080873",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097102",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090780",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791469680",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090533",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654080895",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097021",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045884",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089973",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792932470=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090054",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654089940",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044274",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654091395",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043703",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043644",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045044",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654096122",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654086860",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045302",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044473",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097986",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041942",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041964",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041850",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043526",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043585",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043574",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043552",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043305",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468711",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468722",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791470295",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654094836",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043110",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654098255",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045206",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043633",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097441",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043364",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044915",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468792",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468700",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791469702",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468674",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044171",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468744",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926800=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468895",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468910",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468921",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468932",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043154",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654043224",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090091",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090102",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654097091",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654090216",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468906",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=183791468755",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654087420",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042045",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041754",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654086856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654041695",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654078014",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654087534",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654045125",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654042410",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654078261",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654078250",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044694",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044716",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044683",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654044764",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655503730",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655504813",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655505432",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507414",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926715=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792919170=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792926365=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792927054=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"https://trackings.post.japanpost.jp/services/srv/search?requestNo1=183792939396=&search.x=62&search.y=13&startingUrlPatten=&locale=ja",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655506191",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507075",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507720",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655506250",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502374",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512826",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512782",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512804",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502400",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502433",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513353",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513283",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513294",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502422",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513364",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507241",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655501976",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655516864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655516886",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517085",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517122",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517144",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517041",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517111",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512686",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502094",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655516982",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517004",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517063",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507355",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502035",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512690",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502190",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517354",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512955",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517310",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502713",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517446",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517380",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517376",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513003",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513110",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655511872",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512174",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655502864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513246",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507871",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513250",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507904",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513235",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513482",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513504",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655512185",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517715",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517726",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655513460",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517741",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517752",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655507941",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655503155",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655508092",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517881",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655517903",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655518146",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655508464",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508570",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508566",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116504462",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508802",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508662",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508765",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506466",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513901",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513573",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506304",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506326",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505103",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508614",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505700",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508636",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508603",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508651",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506341",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508625",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508905",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505081",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505136",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505092",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508581",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513820",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508673",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508706",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505766",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508684",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508721",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508743",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513680",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513481",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507096",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513702",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513470",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507155",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506923",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512840",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509340",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513304",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508791",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513536",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513562",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509583",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513746",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508496",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513761",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507240",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507262",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509981",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116511856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513385",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513735",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505383",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509885",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513794",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509863",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510331",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513610",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508813",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508835",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506595",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510714",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506606",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506584",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508916",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506562",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512221",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506632",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505070",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506374",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116511930",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506363",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506654",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506665",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512766",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509174",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507435",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506676",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507413",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508990",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509001",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509303",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506400",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509944",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116511506",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509196",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509723",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513282",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507472",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506746",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506750",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507483",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506890",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506886",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506875",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507063",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507450",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507505",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509734",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509745",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506433",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510305",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510272",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507612",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512626",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512895",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512910",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512906",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512873",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513326",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509373",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505851",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506411",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509362",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512593",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507122",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509336",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509200",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506901",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508286",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509351",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509454",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506503",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510320",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509756",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510025",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510316",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510294",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509852",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509502",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510121",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512081",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509480",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509476",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509771",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509491",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509535",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509874",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116514074",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116514085",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509572",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116514052",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510110",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508301",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116514063",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512980",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116511941",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507306",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506094",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507310",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512744",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506061",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512184",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506013",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512976",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506142",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509395",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513234",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509933",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513223",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116504053",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509292",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509185",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509421",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509432",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507284",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506971",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509594",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512630",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509605",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116504075",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509653",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509664",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509830",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510095",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509233",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509524",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509815",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509631",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509620",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509826",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510040",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509616",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509465",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512044",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509244",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509782",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509896",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510143",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509266",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509675",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116513190",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512711",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510703",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507645",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512722",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510725",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508334",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506024",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510401",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507940",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506293",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507951",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507973",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507660",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507671",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512475",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507962",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116505674",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510806",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507995",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507984",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512770",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508065",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507321",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508032",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508043",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508021",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506105",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508006",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508010",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508080",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508382",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512486",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508076",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508393",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508113",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507343",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512490",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507332",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507354",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507030",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508146",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508161",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509911",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507074",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508220",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507446",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508986",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509922",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507052",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507041",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508172",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507494",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508216",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116512814",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508205",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507376",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508135",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507365",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508124",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507166",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509712",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507586",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507553",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508231",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507542",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116508242",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507575",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509325",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116507520",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506945",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506982",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116506934",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116509550",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116510106",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655493392",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655495046",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655495256",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655495315",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655496085",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655500624",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655493775",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655494276",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655494792",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655504161",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655505443",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655505480",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655506106",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655509982",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654646230",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654646171",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654644292",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654644023",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654646366",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654643710",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654642623",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654643802",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654630325",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654608990",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654615076",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654632112",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654632112",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654630325",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654642096",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654642413",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654641794",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654605582",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654612206",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654621741",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654612626",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654612184",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654605582",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596880",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654621354",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596876",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654615076",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596902",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654612221",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654612136",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654621741",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654621354",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610655",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654634083",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596143",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626615",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610563",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610600",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654604333",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626276",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610423",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654618016",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654618016",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610471",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654618053",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654618053",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610530",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654637841",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613315",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654604333",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628310",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628590",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613061",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654637841",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610412",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613002",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595141",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654602734",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654602734",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654603040",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654603040",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595384",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609896",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625160",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625171",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624784",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609196",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594485",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594570",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594474",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654615345",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595196",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654615345",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610902",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610950",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625086",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654618683",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596305",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610773",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610843",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654618683",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596213",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626851",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595480",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625392",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609841",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609970",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625451",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609922",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595476",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609944",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595152",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627352",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654620820",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654614004",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627315",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654614004",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623966",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654620820",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624434",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624401",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624445",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594706",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624703",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623631",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654630642",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613680",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654630642",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613536",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623631",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597064",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611440",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611425",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627724",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593785",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627271",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611506",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611963",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611436",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596714",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596736",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596924",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596740",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596681",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611160",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611134",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611193",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626604",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596224",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627050",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626685",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654634960",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626593",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654634960",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626313",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626921",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626394",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654596025",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595830",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625952",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595793",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624950",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609480",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597112",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594931",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597333",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595001",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654631305",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654631305",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597182",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624666",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597296",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594894",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624552",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654601802",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654601802",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627374",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627245",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627385",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611694",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611171",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594511",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654635450",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654635450",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624342",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611565",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594555",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626965",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624106",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627293",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654627260",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611300",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594544",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654611285",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654600096",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594135",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624121",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654595885",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594264",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654600096",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654614601",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654614601",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617681",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654626162",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617681",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610401",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624176",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654594032",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610316",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617456",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617456",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617202",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654610143",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654604031",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654608754",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617202",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654624025",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654604031",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654629566",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654629566",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625543",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654608430",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625834",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654617084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625425",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623885",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654608323",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625812",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654625646",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654629765",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654629765",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623900",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623896",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593284",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654609690",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613724",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613724",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593774",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623745",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654608172",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593531",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613746",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613746",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593483",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593590",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623852",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613363",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597646",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628940",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628376",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654599201",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628262",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597506",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628424",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593321",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654593251",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597510",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628715",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628796",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628903",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597650",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654623841",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654599201",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597786",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628041",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597392",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597495",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654637314",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654606470",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654637336",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654637336",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597381",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597101",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654606470",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613374",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597904",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654637314",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597355",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597882",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613525",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654613610",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654628892",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654278892",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654278892",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654278892",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483654597834",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655555246",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655550851",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655555460",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551013",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551186",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551352",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551470",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551794",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556436",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655552726",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553135",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553290",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554082",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554093",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554266",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554745",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554476",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554911",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553382",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553496",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553791",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553953",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553894",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553522",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553920",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655554550",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655550663",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551164",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655551864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655553544",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655552063",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655552715",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541946",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655558186",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655558654",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655542296",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655542521",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655558960",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655558562",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655558853",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655542790",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655559166",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655542952",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655559133",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655548563",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655559494",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655549064",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655559940",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655559811",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655565536",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560581",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560636",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655565562",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560824",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560791",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560802",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560625",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655565746",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655543840",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655566472",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655561034",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655544584",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655561454",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655544816",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655561690",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655561675",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655544864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655562014",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655545634",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655562493",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655562401",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655562423",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655562655",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655562806",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655559564",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655563101",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655563790",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655563893",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655547126",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655548191",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655544772",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655563064",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655546043",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655545855",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655549650",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556720",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556720",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556775",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556775",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556694",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556694",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556801",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556801",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655540616",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655540616",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556952",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556952",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556904",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556904",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556963",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556963",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541073",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541073",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556882",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655556882",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557291",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557291",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557210",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557276",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541482",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557431",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541585",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541563",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557523",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557490",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541725",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655557700",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655541902",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655560931",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655545601",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655546183",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655564254",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655565282",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655566030",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655550206",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655566181",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655550210",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=483655550405",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116516804",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517110",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116516432",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520606",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517084",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517095",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116515872",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116518845",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520551",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116516340",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116516174",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116516546",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520735",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520702",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520746",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517773",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519394",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520072",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520503",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116518860",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116516896",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517051",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519335",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520595",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520584",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519593",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520676",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517401",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520665",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520293",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116517563",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519162",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519151",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520643",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520654",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520282",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520341",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116518694",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519350",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519615",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519626",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519604",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519641",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519630",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519755",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519700",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519711",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519696",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519685",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519663",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519652",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519733",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519744",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519906",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520094",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520083",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116519943",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520105",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520131",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520186",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520164",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520540",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520153",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520201",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520190",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520234",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520212",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520260",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520256",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520632",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520610",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520691",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520680",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520761",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116520783",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574495",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567296",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567436",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567005",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567090",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567016",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567064",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567020",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566493",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566751",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566810",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566434",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566740",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566526",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566574",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566832",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566585",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574554",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567462",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567871",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566445",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567462",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566854",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567705",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573773",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567230",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566961",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573740",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573751",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573504",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573515",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567753",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573530",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573526",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573806",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573795",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567764",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573810",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573563",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574296",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574311",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574005",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573843",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573891",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574285",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573913",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574333",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574370",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574016",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573946",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567484",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574042",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574031",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574020",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566876",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573600",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574392",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567495",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567506",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574053",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574156",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116572675",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566880",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574160",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574075",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567510",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574086",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116572686",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573692",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567580",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566913",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574425",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566924",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574436",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573036",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566946",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116572756",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573622",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567591",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573644",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116572723",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574090",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116572745",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574215",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574226",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574112",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116572804",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573725",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573950",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567624",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573666",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116574101",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573655",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116573736",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567775",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567031",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567812",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567451",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567790",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567554",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116567856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116566423",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581134",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579970",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580272",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580261",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579944",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580843",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579955",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580880",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579815",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579981",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579830",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580902",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580891",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582162",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582173",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582276",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581985",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581554",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583002",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582674",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581764",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581952",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581543",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581521",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583072",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581333",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583061",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581635",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581602",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583050",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583035",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581576",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581786",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582700",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581565",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582011",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582534",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583094",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582022",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581856",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582770",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581650",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582125",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582906",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585312",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580596",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580375",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579782",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579933",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580143",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582206",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580935",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585264",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584903",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580924",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582501",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116579863",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580670",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580692",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582405",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580961",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583175",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582582",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580994",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583164",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582571",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581031",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582744",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581090",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580364",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581506",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116580655",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584306",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583422",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584111",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581215",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584133",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583400",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584144",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584741",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582490",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581322",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582383",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581812",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581624",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582361",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581801",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582033",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582420",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581366",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582545",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582556",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582066",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116586200",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583444",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583466",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581381",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584310",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582151",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583455",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584166",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583514",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583621",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583816",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584365",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584030",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584866",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584214",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584343",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584203",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584192",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584321",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584170",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584181",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585640",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581193",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585754",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582210",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584240",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582221",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581160",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585673",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581930",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584041",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585721",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584052",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585651",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583831",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581484",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116581204",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583842",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584262",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584273",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116582335",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584376",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585776",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583665",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583676",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584391",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583923",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584472",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583853",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585511",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583901",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584461",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583654",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583702",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583691",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583945",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583934",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116586174",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583551",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585986",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584575",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583540",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116586104",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583595",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584085",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583864",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583525",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585850",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583584",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583724",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583960",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116585426",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583750",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584612",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584590",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584601",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583610",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583971",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116586163",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584623",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116586185",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584774",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583573",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116583761",
"http://jizen.kuronekoyamato.co.jp/jizen/servlet/crjz.b.NQ0010?id=507116584634"
]

# ========= 循环发送请求 =========
responses = []

for idx, url in enumerate(START_URLS, start=1):
    payload = {
        "sitemap_id": SITEMAP_ID,
        "driver": "fulljs",               # 或 "fulljs"
        "page_load_delay": 2000,
        "request_interval": 2000,
        "proxy": "datacenter-us",
        "start_urls": [url],            # 每次只发一个
        "custom_id": f"custom-job-{idx}"
    }

    print(f"[{idx}] Sending scraping job for:", url)

    resp = requests.post(
        API_URL,
        params={"api_token": API_TOKEN},
        json=payload,
        headers=HEADERS,
        timeout=30
    )

    responses.append(resp)

    print("Status:", resp.status_code)
    print("Response:", resp.text)
    print("-" * 60)

    # 防止触发 API 限流（可按需要调整）
    time.sleep(6)

In [4]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry




def query_yamato_tracking(tracking_numbers):
    """
    查询黑猫宅急便追踪信息

    Args:
        tracking_numbers: 追踪号列表 (最多10个)

    Returns:
        dict: 查询结果 (HTML响应)
    """

    url = "https://toi.kuronekoyamato.co.jp/cgi-bin/tneko"

    # 构建表单数据
    data = {
        "mypagesession": "",
        "backaddress": "",
        "backrequest": "get",
    }

    # 添加追踪号
    for i, number in enumerate(tracking_numbers[:10], 1):
        data[f"number{i:02d}"] = number

    # 填充剩余字段
    for i in range(len(tracking_numbers) + 1, 11):
        data[f"number{i:02d}"] = ""

    # 配置会话
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    # 发送请求
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
        "Referer": "https://toi.kuronekoyamato.co.jp/cgi-bin/tneko"
    }

    response = session.post(url, data=data, headers=headers)
    return response.text


result = query_yamato_tracking(["4836-5538-3061", "4836-5538-3050"])
print(result)

SSLError: HTTPSConnectionPool(host='toi.kuronekoyamato.co.jp', port=443): Max retries exceeded with url: /cgi-bin/tneko (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] ssl/tls alert handshake failure (_ssl.c:1007)')))

In [5]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context
import ssl
import certifi

class RobustHTTPAdapter(HTTPAdapter):
    """处理SSL/TLS握手问题的强化适配器"""

    def init_poolmanager(self, *args, **kwargs):
        ctx = create_urllib3_context()

        # 配置TLS版本范围
        ctx.minimum_version = ssl.TLSVersion.TLSv1_2
        ctx.maximum_version = ssl.TLSVersion.TLSv1_3

        # 配置密码套件
        ctx.set_ciphers('ECDHE+AESGCM:ECDHE+CHACHA20:DHE+AESGCM:DHE+CHACHA20:!aNULL:!eNULL:!EXPORT:!DSS:!DES:!RC4:!3DES:!MD5:!PSK')

        # 使用系统CA证书
        ctx.load_verify_locations(certifi.where())

        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

# 创建会话
session = requests.Session()
session.mount('https://', RobustHTTPAdapter())
session.mount('http://', HTTPAdapter())

def query_yamato(tracking_numbers):
    """查询黑猫宅急便追踪信息"""

    url = "https://toi.kuronekoyamato.co.jp/cgi-bin/tneko"

    # 构建表单数据
    data = {
        "mypagesession": "",
        "backaddress": "",
        "backrequest": "get",
    }

    for i, number in enumerate(tracking_numbers[:10], 1):
        data[f"number{i:02d}"] = number

    for i in range(len(tracking_numbers) + 1, 11):
        data[f"number{i:02d}"] = ""

    # 完整的请求头
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'ja-JP,ja;q=0.9,zh-CN;q=0.8,zh;q=0.7,en;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://toi.kuronekoyamato.co.jp/cgi-bin/tneko',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0'
    }

    try:
        response = session.post(
            url,
            data=data,
            headers=headers,
            timeout=15,
            verify=True,  # 验证SSL证书
            allow_redirects=True
        )
        response.raise_for_status()
        return response
    except requests.exceptions.SSLError as e:
        print(f"SSL Error: {e}")
        # 降级到较宽松的SSL验证
        response = session.post(
            url,
            data=data,
            headers=headers,
            timeout=15,
            verify=certifi.where(),  # 使用certifi的证书包
            allow_redirects=True
        )
        return response
    except Exception as e:
        print(f"Error: {e}")
        raise

# 使用示例

tracking_numbers = ["4836-5538-3061"]

try:
    response = query_yamato(tracking_numbers)
    print(f"HTTP Status: {response.status_code}")
    print(f"Content Length: {len(response.text)}")
    print("\\n--- Response Preview ---")
    print(response.text[:1000])
except Exception as e:
    print(f"Failed: {e}")

SSL Error: HTTPSConnectionPool(host='toi.kuronekoyamato.co.jp', port=443): Max retries exceeded with url: /cgi-bin/tneko (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] ssl/tls alert handshake failure (_ssl.c:1007)')))
Failed: HTTPSConnectionPool(host='toi.kuronekoyamato.co.jp', port=443): Max retries exceeded with url: /cgi-bin/tneko (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] ssl/tls alert handshake failure (_ssl.c:1007)')))


In [99]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context
import ssl

# 自定义SSL上下文
class CustomHTTPAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = create_urllib3_context(
            ssl_version=ssl.PROTOCOL_TLS,  # 自动选择最佳版本
            ciphers='DEFAULT'
        )
        # 禁用某些检查
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_REQUIRED
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

# 使用自定义适配器
session = requests.Session()
session.mount('https://', CustomHTTPAdapter())

url = "https://toi.kuronekoyamato.co.jp/cgi-bin/tneko"
data = {
    "mypagesession": "",
    "backaddress": "",
    "backrequest": "get",
    "number01": "4836-5538-3050",
    "number02": "4836-5538-3061",
    "number03": "4836-5550-4161",
    "number04": "4836-5550-5443",
    "number05": "4836-5550-5480",
    "number06": "4836-5550-6106",
    "number07": "4836-5550-9982",
    "number08": "4836-5464-6230",
    "number09": "4836-5464-6171",
    "number10": "4836-5464-4292"
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'ja-JP,ja;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://toi.kuronekoyamato.co.jp/cgi-bin/tneko',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

try:
    response = session.post(url, data=data, headers=headers, timeout=10)
    print(f"Status: {response.status_code}")
    print(response.text)
except Exception as e:
    print(f"Error: {e}")

print(response.content)

Status: 200
<!DOCTYPE HTML>
<html lang="ja">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width">
<meta name="format-detection" content="telephone=no">
<title>荷物お問い合わせシステム</title>
<link rel="stylesheet" href="/css/style.css?20240723001">
<link rel="stylesheet" href="/css/important.css?20230602001">
<script src="/js/jquery-3.5.1.min.js?20210729001"></script>
<script src="/js/base.js?20230611001"></script>

<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-TH8FNK5');</script>
<!-- End Google Tag Manager -->

<script language="JavaScript">
<!--
var w = screen.availWidth;
var h = screen.availHeigh

In [98]:
from lxml import etree
from datetime import datetime
import re

def extract_tracking_data(html_content: str, year: int = 2026):
    parser = etree.HTMLParser()
    tree = etree.fromstring(html_content, parser)

    results = []

    # 每一行：div.tracking-box-area
    rows = tree.xpath("//div[contains(@class,'tracking-box-area')]")
    for row in rows:
        # 送り状番号（value 里是 4836-5538-3050 这种）
        value = "".join(row.xpath(".//div[contains(@class,'data') and contains(@class,'number')]//input/@value")).strip()
        digits = re.sub(r"\D", "", value)  # 去掉 '-'
        tracking_number = digits if len(digits) >= 12 else None  # 通常正好 12 位

        # 日付（PC: div.data.date.pc-only）
        date_text = "".join(row.xpath(".//div[contains(@class,'data') and contains(@class,'date') and contains(@class,'pc-only')]/text()")).strip()
        delivery_date = None
        if date_text:
            try:
                delivery_date = datetime.strptime(f"{year}/{date_text}", "%Y/%m/%d").date()
            except ValueError:
                delivery_date = None

        # 配送状況：<a> 内文本包含“01/13”与“配達完了”，这里提取最后的状态词
        a = row.xpath(".//div[contains(@class,'data') and contains(@class,'state')]//a")
        delivery_status = None
        if a:
            full_text = "".join(a[0].itertext())
            full_text = re.sub(r"\s+", " ", full_text).strip()

            # 去掉类似 "01/13"、"01/13　" 这种日期片段，保留状态
            full_text = re.sub(r"\b\d{2}/\d{2}\b", "", full_text).strip()
            # 如果还残留箭头符号或多余空格，继续清理
            full_text = full_text.replace("▶", "").strip()
            delivery_status = full_text or None  # 通常最终是 "配達完了"

        if tracking_number or delivery_date or delivery_status:
            results.append({
                "tracking_number": tracking_number,
                "delivery_date": delivery_date,
                "delivery_status": delivery_status,
                # 如需保留原始格式（含 -），打开下面一行
                # "tracking_number_raw": value or None,
            })

    return results
if __name__ == '__main__':
    data = extract_tracking_data(response.text)

    for i, item in enumerate(data, 1):
        print(f"Item {i}:")
        print(f" Tracking Number: {item['tracking_number']}")
        print(f" Delivery Date: {item['delivery_date']}")
        print(f" Delivery Status: {item['delivery_status']}")
        print()

    print("=" * 10)

    print(data)

Item 1:
 Tracking Number: 483655383050
 Delivery Date: 2026-01-13
 Delivery Status: 配達完了

Item 2:
 Tracking Number: 483655383061
 Delivery Date: 2026-01-13
 Delivery Status: 配達完了

Item 3:
 Tracking Number: 483655504161
 Delivery Date: 2026-01-14
 Delivery Status: 配達完了

Item 4:
 Tracking Number: 483655505443
 Delivery Date: 2026-01-14
 Delivery Status: 配達完了

Item 5:
 Tracking Number: 483655505480
 Delivery Date: 2026-01-14
 Delivery Status: 配達完了

Item 6:
 Tracking Number: 483655506106
 Delivery Date: 2026-01-14
 Delivery Status: 配達完了

Item 7:
 Tracking Number: 483655509982
 Delivery Date: 2026-01-14
 Delivery Status: 配達完了

Item 8:
 Tracking Number: 483654646230
 Delivery Date: 2026-01-09
 Delivery Status: 配達完了

Item 9:
 Tracking Number: 483654646171
 Delivery Date: 2026-01-09
 Delivery Status: 配達完了

Item 10:
 Tracking Number: 483654644292
 Delivery Date: 2026-01-09
 Delivery Status: 配達完了

[{'tracking_number': '483655383050', 'delivery_date': datetime.date(2026, 1, 13), 'delivery_status'

In [128]:
import os

ROOT = "9/"
EXCEL_EXTS = {".json",}
IGNORE_DIRS = {".git", ".idea", "__pycache__", "node_modules", ".venv", "venv"}

def fast_list(root: str):
    files, parents = [], set()
    for dirpath, dirnames, filenames in os.walk(root):
        # 剪枝：从待遍历子目录中移除忽略目录
        dirnames[:] = [d for d in dirnames if d not in IGNORE_DIRS]
        for fn in filenames:
            if fn.startswith("~$"):
                continue
            ext = os.path.splitext(fn)[1].lower()
            full = os.path.abspath(os.path.join(dirpath, fn))
            files.append(full)
            parents.add(os.path.dirname(full))
    files.sort()
    return files, sorted(parents)


files, parent_dirs = fast_list(ROOT)
print(files)

['/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:39.853+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:51.813+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:54.794+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:57.672+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:00.193+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:02.542+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:05.485+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:06.093+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:08.286+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:09.277+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:11.170+09:00-a', '/Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:12.113+09:00-a', '/Users/syu/PycharmProjects

In [129]:
API_URL = "http://data.yamaguchi.lan/api/aggregation/emails/ingest/"
API_TOKEN = "FAZEHBZu0g2o3sRfQ58MxRC0w0htdUoPaLDN8R3ku8dJxk5exDgEUC1GtbJhwWWJKr4s8E"
import json
import time
import traceback
from typing import List, Dict, Any, Optional

import requests
from requests import Response
CA_BUNDLE = "/Users/syu/caddy-local-root-ca.crt"   # 这里换成你拷贝出来的 Caddy Root CA 路径

def load_emails_from_json(filepath: str) -> List[Dict]:
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    if isinstance(data, list):
        return data
    if isinstance(data, dict):
        return [data]
    raise ValueError("Invalid JSON format: expected list or dict")


def _safe_response_text(resp: Optional[Response], limit: int = 2000) -> str:
    if resp is None:
        return ""
    try:
        text = resp.text
    except Exception:
        return "<failed to read response text>"
    if text is None:
        return ""
    return text[:limit] + ("...(truncated)" if len(text) > limit else "")


def batch_ingest_emails(emails: List[Dict]) -> Dict[str, Any]:
    headers = {
        "Authorization": f"Bearer {API_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {"emails": emails}

    try:
        resp = requests.post(
            API_URL,
            json=payload,
            headers=headers,
            timeout=10,
            verify=CA_BUNDLE,  # 关键：告诉 requests 信任你的内网 CA
        )
        # 先处理非 2xx，方便打印响应体
        if not resp.ok:
            body_preview = _safe_response_text(resp)
            raise requests.HTTPError(
                f"HTTP {resp.status_code} Error. Response preview: {body_preview}",
                response=resp,
            )

        # 尝试解析 JSON；如果不是 JSON，也要给出可读信息
        try:
            return resp.json()
        except ValueError:
            body_preview = _safe_response_text(resp)
            raise ValueError(f"Response is not valid JSON. Response preview: {body_preview}")

    except requests.RequestException as e:
        # 把尽可能多的上下文打印出来，然后把异常继续抛给 main 去决定“跳过”
        resp = getattr(e, "response", None)
        print("\n[ERROR] batch_ingest_emails 请求失败")
        print(f"  API_URL: {API_URL}")
        print(f"  Exception: {repr(e)}")
        if resp is not None:
            print(f"  HTTP Status: {resp.status_code}")
            print(f"  Response preview: {_safe_response_text(resp)}")
        # 如果你想更详细，可打开下一行
        # traceback.print_exc()
        raise


def main(filepath: str) -> bool:
    """
    返回 True 表示该文件导入成功（至少请求成功并拿到结果），False 表示该文件处理失败但已记录错误。
    """
    try:
        emails = load_emails_from_json(filepath)
        print(f"从文件加载了 {len(emails)} 封邮件: {filepath}")

        result = batch_ingest_emails(emails)

        print("导入结果:")
        print(f"  状态: {result.get('status')}")
        print(f"  总计: {result.get('total')} 封")
        print(f"  成功: {result.get('successful')} 封")
        print(f"  失败: {result.get('failed')} 封")

        # 打印单条明细（如果有）
        if result.get("results"):
            failed_items = [x for x in result["results"] if x.get("status") != "success"]
            if failed_items:
                print("失败明细:")
                for item in failed_items[:50]:  # 避免过多输出
                    print(f"  ✗ {item.get('email_id')} -> {item.get('error', 'Unknown')}")
                if len(failed_items) > 50:
                    print(f"  ...(truncated) 还有 {len(failed_items)-50} 条失败未显示")

        return True

    except Exception as e:
        print("\n[ERROR] 处理文件失败，已跳过继续执行")
        print(f"  File: {filepath}")
        print(f"  Exception: {repr(e)}")
        # 打印栈信息，便于定位是 JSON 解析问题还是网络/证书/服务端错误
        traceback.print_exc()
        print("-" * 80)
        return False


# 你的原始逻辑不变，只是 main 返回 False 时继续即可
ind = 0

success_count = 0
fail_count = 0

for index, filepath in enumerate(files[ind:], start=0):
    print(index, "  --", filepath)
    ok = main(filepath)
    if ok:
        success_count += 1
    else:
        fail_count += 1
    time.sleep(1)

print(f"\n批处理完成：成功 {success_count} 个文件，失败 {fail_count} 个文件")

0   -- /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:39.853+09:00-a
从文件加载了 3 封邮件: /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:39.853+09:00-a
导入结果:
  状态: success
  总计: 3 封
  成功: 3 封
  失败: 0 封
1   -- /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:51.813+09:00-a
从文件加载了 10 封邮件: /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:51.813+09:00-a
导入结果:
  状态: success
  总计: 10 封
  成功: 10 封
  失败: 0 封
2   -- /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:54.794+09:00-a
从文件加载了 10 封邮件: /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:54.794+09:00-a
导入结果:
  状态: success
  总计: 10 封
  成功: 10 封
  失败: 0 封
3   -- /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:57.672+09:00-a
从文件加载了 10 封邮件: /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:50:57.672+09:00-a
导入结果:
  状态: success
  总计: 10 封
  成功: 10 封
  失败: 0 封
4   -- /Users/syu/PycharmProjects/YamagotiProjects/9/2026-01-24T11:51:00.193+09:00-

In [67]:
from lxml import html
import re
# 读取HTML文件
with open('2.html', 'r', encoding='utf-8') as f:
    content = f.read()

tree = html.fromstring(content)

order_link_element = tree.xpath("//a[@class='aapl-link' and contains(@href, 'vieworder')]")[0]
order_text = order_link_element.xpath('.//text()')[0]
order_href = order_link_element.xpath('./@href')[0]

print(f"订单号文本: {order_text}")
print(f"订单号链接: {order_href}")


# 2. 获取订单日期 2026/01/13（文本）
order_date = tree.xpath("//div[@class='order-num']//span[contains(text(), '2026/01/')]/text()")[0]
print(f"订单日期: {order_date}")



with open('2.html', 'r', encoding='utf-8') as f:
    tree = html.fromstring(f.read())

# ========== 提取配送日期（支持单一日期和日期范围）==========
def extract_delivery_dates():
    """支持两种格式的配送日期提取（更强大的版本）"""

    # 方法1: 查找所有包含日期格式的文本
    all_text = ' '.join(tree.xpath("//text()"))

    # 使用正则表达式找出所有日期
    date_pattern = r'\\d{4}/\\d{2}/\\d{2}'
    all_dates = re.findall(date_pattern, all_text)

    if not all_dates:
        return None

    # 方法2: 查找"お届け予定日"后面的日期
    delivery_section = tree.xpath("//*[contains(text(), 'お届け予定日')]")

    if delivery_section:
        # 获取该元素及其后续所有文本
        delivery_text = ''.join(
            tree.xpath("//*[contains(text(), 'お届け予定日')]/following::text()[position() < 20]")
        )
        delivery_dates = re.findall(date_pattern, delivery_text)

        if delivery_dates:
            if len(delivery_dates) >= 2:
                return {
                    'type': 'range',
                    'start_date': delivery_dates[0],
                    'end_date': delivery_dates[1],
                    'full_text': f"{delivery_dates[0]} - {delivery_dates[1]}"
                }
            elif len(delivery_dates) == 1:
                return {
                    'type': 'single',
                    'date': delivery_dates[0],
                    'full_text': delivery_dates[0]
                }

    return None



delivery_info = extract_delivery_dates()
print(f"配送日期信息: {delivery_info}")

if delivery_info['type'] == 'range':
    print(f"开始日期: {delivery_info['start_date']}")
    print(f"结束日期: {delivery_info['end_date']}")
else:
    print(f"配送日期: {delivery_info['date']}")


# 4. 获取产品名称
product_name = tree.xpath("//td[@class='product-name-td']//text()[normalize-space()]")[0].strip()
print(f"产品名称: {product_name}")

# 5. 获取数量中的 1
quantity = tree.xpath("//td[@class='product-quantity']//text()")[0].strip()
print(f"数量: {quantity}")

# 6. 获取配送地址相关信息
# 获取所有gen-txt类的div（这是地址部分）
address_divs = tree.xpath("//h3[contains(text(), 'お届け先住所')]/following-sibling::div[@class='gen-txt']/text()")

postal_code = address_divs[0].strip()  # 104-0041
prefecture = address_divs[1].strip()   # 東京都中央区
address = address_divs[2].strip()      # 新富1-9-6-101-ybo805t
recipient_name = address_divs[3].strip().replace('様', '')  # 小関 優斗

print(f"邮编: {postal_code}")
print(f"都道府县: {prefecture}")
print(f"地址: {address}")
print(f"收货人: {recipient_name}")

# 7. 获取邮箱地址
email = tree.xpath("//h3[contains(text(), '商品出荷のお知らせ')]/following-sibling::div//text()[contains(., '@')]")[0].strip()
print(f"邮箱: {email}")

订单号文本: W1626439776
订单号链接: https://store.apple.com/go/jp/vieworder//W1626439776/wqx378@yamaguchishoji.com/AOS-JP-A10000085033
订单日期: 2026/01/10
配送日期信息: None


TypeError: 'NoneType' object is not subscriptable

In [75]:
from lxml import html
import re
from typing import Optional, Dict, Any


with open('499.html', 'r', encoding='utf-8') as f:
    tree = html.fromstring(f.read())

# ========== 提取配送日期（支持单一日期和日期范围）==========

def extract_delivery_dates(tree) -> Optional[Dict[str, Any]]:
    """支持两种格式的配送日期提取（区间 / 单日），并兼容日期在文本节点或子标签内"""

    # 只匹配带冒号的那一个（避免误匹配上面“お届け予定日 1”的标题）
    spans = tree.xpath(
        "//span[contains(normalize-space(.), 'お届け予定日') and (contains(., '：') or contains(., ':'))]"
    )
    if not spans:
        return None

    span = spans[0]
    container = span.getparent()  # <span> 所在的父节点，通常就是那个包着日期的 <div>

    # 抓取容器内的全部文本（包含子元素内文本），避免日期被包在其他标签里导致漏抓
    raw_text = "".join(container.xpath(".//text()"))
    normalized = re.sub(r"\s+", " ", raw_text).strip()

    # 只从“お届け予定日：”之后开始提取日期，避免容器内其他位置存在日期时误抓
    m = re.search(r"お届け予定日\s*[:：]\s*", normalized)
    target_text = normalized[m.end():] if m else normalized

    dates = re.findall(r"\d{4}/\d{2}/\d{2}", target_text)
    if not dates:
        return None

    if len(dates) >= 2:
        return {
            "type": "range",
            "start_date": dates[0],
            "end_date": dates[1],
            "full_text": f"{dates[0]} - {dates[1]}",
        }

    return {
        "type": "single",
        "date": dates[0],
        "full_text": dates[0],
    }

# ========== 订单号（文本+链接）==========
order_links = tree.xpath("//a[@class='aapl-link' and contains(@href, 'vieworder')]")
if order_links:
    order_link = order_links[0]
    order_number = {
        'text': order_link.xpath('./text()')[0] if order_link.xpath('./text()') else '',
        'href': order_link.xpath('./@href')[0] if order_link.xpath('./@href') else ''
    }
else:
    order_number = None

# ========== 订单日期 ==========
order_dates = tree.xpath("//div[@class='order-num']//span[contains(text(), '/')]/text()")
order_date = order_dates[0] if order_dates else None

# ========== 配送日期 ==========
delivery_info = extract_delivery_dates(tree)

# ========== 产品名称 ==========
product_names = tree.xpath("//td[@class='product-name-td']//text()")
product_name = product_names[0].strip() if product_names else None

# ========== 数量 ==========
quantities = tree.xpath("//td[@class='product-quantity']//text()")
quantity = quantities[0].strip() if quantities else None

# ========== 配送地址 ==========
address_elements = tree.xpath("//h3[contains(text(), 'お届け先住所')]/following-sibling::div[@class='gen-txt']/text()")
if len(address_elements) >= 4:
    delivery_address = {
        'postal_code': address_elements[0].strip(),
        'prefecture': address_elements[1].strip(),
        'address': address_elements[2].strip(),
        'recipient': address_elements[3].strip()
    }
else:
    delivery_address = None

# ========== 邮箱 ==========
emails = tree.xpath("//div[contains(., '@')]/text()")
email = None
for e in emails:
    if '@' in e:
        email = e.strip()
        break

# ========== 打印所有数据 ==========
print("===== 订单信息 =====")
print(f"订单号: {order_number['text'] if order_number else 'N/A'}")
if order_number:
    print(f"订单链接: {order_number['href']}")
print(f"订单日期: {order_date}")
print()

print("===== 配送信息 =====")
if delivery_info:
    print(f"配送日期类型: {delivery_info['type']}")
    if delivery_info['type'] == 'range':
        print(f"  开始日期: {delivery_info['start_date']}")
        print(f"  结束日期: {delivery_info['end_date']}")
    else:
        print(f"  配送日期: {delivery_info['date']}")
else:
    print("配送日期: 未找到")
print()

print("===== 产品信息 =====")
print(f"产品名称: {product_name}")
print(f"数量: {quantity}")
print()

print("===== 收货信息 =====")
if delivery_address:
    print(f"邮编: {delivery_address['postal_code']}")
    print(f"都道府县: {delivery_address['prefecture']}")
    print(f"地址: {delivery_address['address']}")
    print(f"收货人: {delivery_address['recipient']}")
else:
    print("地址: 未找到")
print()

print("===== 联系信息 =====")
print(f"邮箱: {email}")

===== 订单信息 =====
订单号: ご注文状況
订单链接: https://store.apple.com/go/jp/vieworder//W1563674370/qde781@yamaguchishoji.com/AOS-JP-A10000085036
订单日期: 2026/01/13

===== 配送信息 =====
配送日期: 未找到

===== 产品信息 =====
产品名称: iPhone 17 Pro Max 512GB コズミックオレンジ
数量: None

===== 收货信息 =====
地址: 未找到

===== 联系信息 =====
邮箱: None


In [78]:
from __future__ import annotations

import json
import re
from typing import Any, Dict, List, Optional

from lxml import html



def xpath_str(tree: html.HtmlElement, expr: str, default: str = "") -> str:
    """
    对于返回 string 的 XPath 表达式：直接拿到字符串；
    对于返回 list 的表达式：取第一个元素并转为字符串。
    """
    res = tree.xpath(expr)
    if isinstance(res, list):
        if not res:
            return default
        v = res[0]
        # v 可能是 element，也可能是 str/bytes/数字
        if hasattr(v, "text_content"):
            return v.text_content().strip()
        return str(v).strip()
    if res is None:
        return default
    return str(res).strip()


def first_group(text: str, pattern: str) -> Optional[str]:
    m = re.search(pattern, text)
    return m.group(1) if m else None


def extract_fields_from_html(html_text: str) -> Dict[str, Any]:
    tree = html.fromstring(html_text)

    # ご注文番号（文本+链接）
    order_number_value = xpath_str(
        tree,
        "normalize-space(string(//span[normalize-space()='ご注文番号:']/following-sibling::span//a))",
        default="",
    ) or None
    order_number_href = xpath_str(
        tree,
        "string(//span[normalize-space()='ご注文番号:']/following-sibling::span//a/@href)",
        default="",
    ) or None

    # ご注文日（文本）
    order_date = xpath_str(
        tree,
        "normalize-space(string(//span[normalize-space()='ご注文日:']/following-sibling::span[1]))",
        default="",
    ) or None

    # お届け予定日（span里包含“お届け予定日 2026/01/19”）
    delivery_blob = xpath_str(
        tree,
        "normalize-space(string(//span[contains(normalize-space(.),'お届け予定日')]))",
        default="",
    )
    delivery_date = first_group(delivery_blob, r"(\d{4}/\d{2}/\d{2})")

    # 商品名
    product_name = xpath_str(
        tree,
        "normalize-space(string(//td[contains(@class,'product-name-td')]))",
        default="",
    ) or None

    # 数量
    qty_blob = xpath_str(
        tree,
        "normalize-space(string(//nobr[contains(normalize-space(.),'数量')]))",
        default="",
    )
    qty_str = first_group(qty_blob, r"(\d+)")
    quantity = int(qty_str) if qty_str is not None else None

    # 配送先住所（多行 div）
    addr_lines: List[str] = [
        s.strip()
        for s in tree.xpath(
            "//h3[contains(normalize-space(.),'配送先住所')]/following-sibling::div[contains(@class,'gen-txt')][1]/div/text()"
        )
        if s and s.strip()
    ]
    postal_code = addr_lines[0] if len(addr_lines) > 0 else None
    prefecture_city = addr_lines[1] if len(addr_lines) > 1 else None
    street_address = addr_lines[2] if len(addr_lines) > 2 else None
    name_line = addr_lines[3] if len(addr_lines) > 3 else None
    recipient_name = re.sub(r"様\s*$", "", name_line).strip() if name_line else None

    # お届け連絡先（邮箱）
    contact_email = xpath_str(
        tree,
        "normalize-space(string(//h3[contains(normalize-space(.),'お届け連絡先')]/following-sibling::div[1]//span[contains(@class,'moe-break-me')]))",
        default="",
    ) or None

    # 配達伝票番号（文本+链接）
    tracking_number = xpath_str(
        tree,
        "normalize-space(string(//h3[contains(normalize-space(.),'配達伝票番号')]/following-sibling::div[1]//a))",
        default="",
    ) or None
    tracking_href = xpath_str(
        tree,
        "string(//h3[contains(normalize-space(.),'配達伝票番号')]/following-sibling::div[1]//a/@href)",
        default="",
    ) or None

    # 配送業者名
    carrier_name = xpath_str(
        tree,
        "normalize-space(string(//h3[contains(normalize-space(.),'配送業者名')]/following-sibling::div[contains(@class,'gen-txt-carrier')][1]))",
        default="",
    ) or None

    return {
        "order_number":order_number_value,
        "official_query_url":order_number_href,
        "confirmed_at": order_date,
        "estimated_website_arrival_date": delivery_date,
        "iphone_product_names": product_name,
        "quantity": quantity,

        "postal_code": postal_code,
        "address_line_1": prefecture_city,
        "address_line_2": street_address,
        "name": recipient_name,

        "email": contact_email,
        "tracking_number": tracking_number,
        "href": tracking_href,
        "carrier_name": carrier_name,
    }


def extract_fields_from_file(path: str) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        return extract_fields_from_html(f.read())



data = extract_fields_from_file("499.html")
print(json.dumps(data, ensure_ascii=False, indent=2))


{
  "order_number": "W1563674370",
  "official_query_url": "https://store.apple.com/go/jp/vieworder//W1563674370/qde781@yamaguchishoji.com/AOS-JP-A10000085037",
  "confirmed_at": "2026/01/13",
  "estimated_website_arrival_date": "2026/01/19",
  "iphone_product_names": "iPhone 17 Pro Max 512GB コズミックオレンジ",
  "quantity": 1,
  "postal_code": "104-0041",
  "address_line_1": "東京都中央区",
  "address_line_2": "新富1-9-6-101-ybo735t",
  "name": "杉原 惟史",
  "email": "qde781@yamaguchishoji.com",
  "tracking_number": "483655967922",
  "href": "https://store.apple.com/go/jp/vieworder//W1563674370/qde781@yamaguchishoji.com/AOS-JP-A10000085036",
  "carrier_name": "YAMATO TRANSPORT CO.,LTD."
}


In [125]:
from datetime import datetime, timedelta
from urllib.parse import urlencode
import requests

# ========= 配置区（你只需要改这里） =========
BASE_WEBHOOK = "https://app-yamaguchi.app.n8n.cloud/webhook/e5ce8962-0721-4864-9a22-debf771fe490"

START_DATE = "2026/1/25"   # X 起始值（YYYY/M/D）
# START_DATE = "2025/12/13"   # X 起始值（YYYY/M/D）
DAYS = 4                  # 循环多少天
DIRECTION = "backward"     # "backward": X 每次 -1 天；"forward": X 每次 +1 天
TIMEOUT = 30               # 请求超时秒数
STOP_ON_FAIL = False       # True：遇到非 2xx 立即停止
# =========================================

def parse_ymd_slash(s: str) -> datetime:
    """解析 YYYY/M/D 或 YYYY/MM/DD"""
    y, m, d = map(int, s.split("/"))
    return datetime(y, m, d)

def fmt_slash(d: datetime) -> str:
    """输出 YYYY/M/D（不补零），与用户示例一致：2026/1/20"""
    return f"{d.year}/{d.month}/{d.day}"

def build_url(before_dt: datetime, after_dt: datetime) -> str:
    params = {"before": fmt_slash(before_dt), "after": fmt_slash(after_dt)}
    return f"{BASE_WEBHOOK}?{urlencode(params)}"

results = []  # 可选：收集结果，后面可转 DataFrame

start_dt = parse_ymd_slash(START_DATE)
step = -1 if DIRECTION == "backward" else 1

with requests.Session() as session:
    for index, i in enumerate(range(DAYS)):
        before_dt = start_dt + timedelta(days=i * step)
        after_dt = before_dt - timedelta(days=1)

        url = build_url(before_dt, after_dt)

        try:
            resp = session.post(url, timeout=TIMEOUT)
            status = resp.status_code
            text_preview = (resp.text or "")[:300].replace("\n", "\\n")
            ok = (200 <= status < 300)

            print(f"[{i+1:04d}] before={fmt_slash(before_dt)} after={fmt_slash(after_dt)} -> HTTP {status} | {text_preview}")

            results.append({
                "i": i + 1,
                "before": fmt_slash(before_dt),
                "after": fmt_slash(after_dt),
                "url": url,
                "status": status,
                "ok": ok,
                "preview": text_preview,
            })

            if STOP_ON_FAIL and not ok:
                print("STOP_ON_FAIL=True，遇到非 2xx，已停止。")
                break

        except requests.RequestException as e:
            print(f"[{i+1:04d}] before={fmt_slash(before_dt)} after={fmt_slash(after_dt)} -> REQUEST ERROR: {e}")
            results.append({
                "i": i + 1,
                "before": fmt_slash(before_dt),
                "after": fmt_slash(after_dt),
                "url": url,
                "status": None,
                "ok": False,
                "preview": str(e),
            })
            if STOP_ON_FAIL:
                print("STOP_ON_FAIL=True，遇到请求异常，已停止。")
                break
        time.sleep(4)
        if (index+1) % 4 == 0:
            time.sleep(60*30)




[0001] before=2026/1/25 after=2026/1/24 -> HTTP 200 | {"message":"Workflow was started"}
[0002] before=2026/1/24 after=2026/1/23 -> HTTP 200 | {"message":"Workflow was started"}
[0003] before=2026/1/23 after=2026/1/22 -> HTTP 200 | {"message":"Workflow was started"}
[0004] before=2026/1/22 after=2026/1/21 -> HTTP 200 | {"message":"Workflow was started"}


KeyboardInterrupt: 

In [115]:
import pandas as pd
import re

def extract_tracking_data(csv_file_path):
    df = pd.read_csv(csv_file_path)
    pattern = r'^\d{4}-\d{4}-\d{4}$'
    mask = df['お問い合わせ番号'].astype(str).str.match(pattern)
    df_filtered = df[mask].copy()
    column_mapping = {
        'お問い合わせ番号': 'tracking_number',
        '最新年月日': 'delivery_status_query_time',
        '最新状態': 'latest_delivery_status',
        'time-scraped': 'last_info_updated_at'
    }
    df_result = df_filtered[list(column_mapping.keys())].rename(columns=column_mapping)
    def clean_text(text):
        if pd.isna(text):
            return None
        return re.sub(r'\s+', ' ', str(text).replace('\n', ' ')).strip()
    if 'delivery_status_query_time' in df_result.columns:
        df_result['delivery_status_query_time'] = df_result['delivery_status_query_time'].apply(clean_text)
    result_list = df_result.to_dict(orient='records')
    return result_list

# 使用例
file_path = 'japan-post-10 (1).csv'
data = extract_tracking_data(file_path)

# 結果の確認（最初の数件を表示）
for item in data[:5]:
    print(item)

{'tracking_number': '1837-9316-7924', 'delivery_status_query_time': '2026/01/18 12:37', 'latest_delivery_status': 'お届け先にお届け済み', 'last_info_updated_at': '2026-01-20 15:47:41'}
{'tracking_number': '1837-9316-7891', 'delivery_status_query_time': '2026/01/18 12:37', 'latest_delivery_status': 'お届け先にお届け済み', 'last_info_updated_at': '2026-01-20 15:47:41'}
{'tracking_number': '1837-9316-7902', 'delivery_status_query_time': '2026/01/18 12:37', 'latest_delivery_status': 'お届け先にお届け済み', 'last_info_updated_at': '2026-01-20 15:47:41'}
{'tracking_number': '1837-9316-7865', 'delivery_status_query_time': '2026/01/18 12:37', 'latest_delivery_status': 'お届け先にお届け済み', 'last_info_updated_at': '2026-01-20 15:47:41'}
{'tracking_number': '1837-9316-7880', 'delivery_status_query_time': '2026/01/18 12:37', 'latest_delivery_status': 'お届け先にお届け済み', 'last_info_updated_at': '2026-01-20 15:47:41'}
